# Exploring Queues
<i>Adapted for use with FABRIC from [Exploring Queues](https://www.cs.unc.edu/Research/geni/geniEdu/09-queues.html)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

This experiment will show the relation between queue size of a throttled router and packet loss/delay. UDP traffic will be sent over a bottlenecked router to develop the target queue.

This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20Queues/Figures/QueueTop.png?raw=true"><br>
In the following experiment we will explore queue behaviour, we will do this by modifying the throughtput of a router node placed between two other nodes and passing traffic between them

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()
import json
import traceback

slice_name = "Queues"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

## 2. Guided Experiment
Note: when modifying the transfer rate using the tc tool, you might have to change the interface depending on the site (from enp7s0 to enp8s0) to see the intended behaviour **the correct interface to modify should be the one connected to the Destination node**
### 2.1 Measuring Loss and Delay
The following steps will show how to measure packet loss and RTT.
1. Open two terminals on the `Source` nodes, one on the `Destination` node and one on the `router` node.

2. In the following experiment we will be measuring two things, the packet loss and the packet delay. Run the following command on the `Destination` node to receive UDP messages:
<br>`iperf -s -u`

3. The first measurement will come from the `Ping` command, it wil be the round trip time (RTT). On one of the `Source` terminal, type but do not run:
<br>`ping 11.1.1.1`

4. On another `Source` terminal, type but do not run:
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10`
<br> This command will send the Destination node UDP traffic at a bandwidth of 20mb/s for 10 seconds.

5. On the source terminals start the Iperf traffic then quickly start the ping command. When the Iperf completes, end the ping with `Ctrl+C`.

6. Your Iperf results should be similar to:
<br>` [  3]  0.0-11.4 sec  23.8 MBytes  17.5 Mbits/sec   0.086 ms	0/17007 (0%) `
<br> The `0/17007 (0%)` shows a 0% packet loss. The ping RTT should be consistent over the duration.

7. Both of these measurement will be used as a measurement of loss they will be used throughout the rest of the experiments be sure you can access them.


### 2.2 Throttling the Router
In step 2.1 it is seen that there is no packet loss and little delay, this is due to the nodes being able to handle more traffic than we were generating (20mbps). Seeing queueing behavior requires the connection to be throttled. we will limit the amount that can be transferred in this segment
1. Go to the `router` terminal and run:
<br>`sudo tc qdisc add dev enp7s0 root tbf rate 13mbit limit 1000mb burst 10kb`
<br>This command will change the outgoing interface to send at a rate of 13mb/s.

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10`
<br> There will be no packet loss on the iperf report, however, there will be an increase in the delay on the ping command during the iperf execution.


### 2.3 Finite Queue
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20Queues/Figures/Queue.png?raw=true"><br>
In 2.2 we saw the formation of a Queue when we limited the traffic that could pass through the router. In this segment we will see the start of packet loss by limiting how large the Queue size can be.

1. Adjust the queue size to 3mb on the `router` node with:
<br>`sudo tc qdisc replace dev enp7s0 root tbf rate 18mbit limit 3mb burst 10kb`

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10` 
<br> There may be some packet loss reported now, in this case the queue was too small for all the traffic.

### 2.4 Smaller Queue
1. Adjust the queue size to 2mb on the `router` node with:
<br>`sudo tc qdisc replace dev enp7s0 root tbf rate 13mbit limit 2mb burst 10kb`

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10`
<br> Modest packet loss should be observed now. On the terminal running ping it can be observed that delay will increase until becoming constant. This occurs when a finite queue fills up, giving it a constant size.

### 2.5 Even Smaller Queue
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20Queues/Figures/dataLoss.png?raw=true"><br>
1. Adjust the queue size to 100kb (.1 mb) on the `router` node with:
<br>`sudo tc qdisc replace dev enp7s0 root tbf rate 7mbit limit 100kb burst 10kb`

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10 &` 
<br> Packet loss will be larger while delay plateaus faster.

## 3. Conclusions
This experiment adjusted the queue size of a throttled router to demonstrate effects on packet loss and delay. We saw that delay gradually increases over time as the queue fills up, eventually reaching a plateau when the queue is entirely full. The peak delay (aka the delay when the queue is entirely full) is directly related to the queue size. This effect is seen clearly in following graph which shows results from the same experiment on some additional queue sizes (100kb 500kb, 1.5mb, 2mb and 3mb).
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20Queues/Figures/Delay%20and%20Queue%20Size.png?raw=true">
<br>We saw a direct correlation between loss and queue size as well. Loss occurs when the queue is not big enough to accommodate the incoming traffic and packets must be dropped. The smaller the queue size, the more dropped packets.
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Exploring%20Queues/Figures/Loss_QueueSize.png?raw=true">
<br>
```
Queue Size (kb)	Delay (ms)	Loss (#packets)
3000				1354			  89
2000				 909			 766
1500				 699			1080
1000				 465			1427
500				  227			1786
100				   45			2054
``` 
## 4. Assignment
### Part 1
The tables and graphs in the conclusion showed the effects of queue size on packet loss and delay under the conditions of the experiment above. Consider re-doing the experiment with 20 seconds of UDP traffic instead of 10 seconds. 
1. How would the peak delay change in the 20 second experiment? 
2. How would the amount of packet loss change in the 20 second experiment? 
Since our 20 second experiment sends twice as much traffic, consider both number of packets and percentage of packets. Please explain your answers.

### Part 2
Redo the experiment with 20 seconds of traffic with the following queue sizes: 3000kb (3mb), 2000kb, 1500kb, 1000kb, 500kb, 100kb. 
1. Find a linear regression line for both peak delay and amount of packet loss. 

Consider your answers from part 1,

2. Do these results match your expectations? Why or why not?

## Cleanup Resources
Once you have completed the assignment shut down the slice

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")